In [1]:
from imports import *
import clean_data_loader as cl

TensorFlow Version: 1.1.0


In [2]:
reviews = pd.read_csv("amazon_dataset/Reviews.csv")
reviews = reviews.dropna()
reviews = reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Score','Time'], 1)
reviews = reviews.reset_index(drop=True)


In [3]:
unclean_summaries=reviews.Summary
unclean_texts=reviews.Text

In [4]:
import preprocessing as pre

def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = pre.clean_text(text) #for any new summary use clean... if already cleaned then skip
    
    return [cl.vocab_to_int.get(word, cl.vocab_to_int['<UNK>']) for word in text.split()]

In [13]:
random = np.random.randint(10,500000)
input_sentence = cl.clean_texts[random]
inp_summ=cl.clean_summaries[random]

text_int = text_to_seq(input_sentence)

print ("\nRANDOM\t=\t=",random)

print ("\n\nRANDOM CHOSEN REVIEW - UNCLEANED - \t=\t",unclean_texts[random])
print ("\nORIGINAL SUMMARY OF THE ABOVE REVIEW - UNCLEANED - \t=\t",unclean_summaries[random])

print ("\n\nRANDOM CHOSEN REVIEW - CLEANED - \t=\t",input_sentence)
print ("\nORIGINAL SUMMARY OF THE ABOVE REVIEW - CLEANED - \t=\t",inp_summ)

#checkpoint = "./best_model.ckpt"

checkpoint ='./model_checkpoints/LSTM_BI_ATTN/best_model.ckpt'
batch_size=32
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    # The input tensors for inference
    input_data = loaded_graph.get_tensor_by_name('input:0')
    inference_logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_seq_len = loaded_graph.get_tensor_by_name('source_seq_len:0')
    target_seq_len = loaded_graph.get_tensor_by_name('target_seq_len:0')
    keep_probability = loaded_graph.get_tensor_by_name('keep_probability:0')

    # Run the graph to get the summary logits
    summary_logits = sess.run(inference_logits, { input_data: [text_int] * batch_size,
                                                  target_seq_len: [np.random.randint(5,8)],
                                                  source_seq_len: [len(text_int)] * batch_size,
                                                  keep_probability: 1.0
                                                })
    # This returns a batch_size - length matrix of logits, so we just need the first one
    summary_logits = summary_logits[0]
    
pad = cl.vocab_to_int["<PAD>"] 


print('  GENERATED SUMMARY\t=\t: {}'.format(" ".join([cl.int_to_vocab[i] for i in summary_logits if i != pad])))
sess.close()


RANDOM	=	= 18366


RANDOM CHOSEN REVIEW - UNCLEANED - 	=	 love the oat meal cooking in cool water add spices eat, will buy again add ripe banana good needs no sugar

ORIGINAL SUMMARY OF THE ABOVE REVIEW - UNCLEANED - 	=	 VERY TASTY  WITH ADDED SPICE


RANDOM CHOSEN REVIEW - CLEANED - 	=	 love oat meal cooking cool water add spices eat buy add ripe banana good needs sugar

ORIGINAL SUMMARY OF THE ABOVE REVIEW - CLEANED - 	=	 very tasty with added spice
INFO:tensorflow:Restoring parameters from ./model_checkpoints/LSTM_BI_ATTN/best_model.ckpt
  GENERATED SUMMARY	=	: great tasting add meal
